# Step 4: Compute 3D wind work on MKE and EKE reservoirs using spatial & temporal filter

This notebook computes the 3D (depth-dependent) diagnostics necessary for making Figure 5 in Loose et al. 2022 (JPO).
* filtered interface height
* wind work on MKE reservoir
* wind work on EKE reservoir

where all three diagnostics above are computed as function of **(layer, y, x)** for (1) a spatial filter and (2) a temporal (500-day) filter. The terms derived with (1) are averaged over 500 days.

In [1]:
filter_fac = 32  # filters 1/32 degree --> 1 degree
end_times = [2500, 2600, 2700, 2800, 2900]  # end times of files that are to be averaged together

In [2]:
import numpy as np
import xarray as xr
from dask.diagnostics import ProgressBar

In [3]:
run = 'nw2_0.03125deg_N15_baseline_hmix20'

### Spatially filtered NeverWorld2 data

In [4]:
scratchpath = '/glade/scratch/noraloose/filtered_data'
nr_days0 = 100  # that's how many days are saved per netcdf

for i, end_time in zip(range(len(end_times)), end_times):
    filename = '%s/%s/averages_%08d_filtered_fac%i' %(scratchpath, run, end_time-nr_days0+2, filter_fac) 
    print(filename)
    av_f_tmp = xr.open_zarr(filename, decode_times=False)

    if i == 0:
        av_f = av_f_tmp
    else:
        av_f = xr.combine_nested([av_f, av_f_tmp], concat_dim='time') 

/glade/scratch/noraloose/filtered_data/nw2_0.03125deg_N15_baseline_hmix20/averages_00002402_filtered_fac32
/glade/scratch/noraloose/filtered_data/nw2_0.03125deg_N15_baseline_hmix20/averages_00002502_filtered_fac32
/glade/scratch/noraloose/filtered_data/nw2_0.03125deg_N15_baseline_hmix20/averages_00002602_filtered_fac32
/glade/scratch/noraloose/filtered_data/nw2_0.03125deg_N15_baseline_hmix20/averages_00002702_filtered_fac32
/glade/scratch/noraloose/filtered_data/nw2_0.03125deg_N15_baseline_hmix20/averages_00002802_filtered_fac32


In [5]:
av_f.time

<xarray.DataArray 'time' (time: 100)>
array([2402.5, 2407.5, 2412.5, 2417.5, 2422.5, 2427.5, 2432.5, 2437.5, 2442.5,
       2447.5, 2452.5, 2457.5, 2462.5, 2467.5, 2472.5, 2477.5, 2482.5, 2487.5,
       2492.5, 2497.5, 2502.5, 2507.5, 2512.5, 2517.5, 2522.5, 2527.5, 2532.5,
       2537.5, 2542.5, 2547.5, 2552.5, 2557.5, 2562.5, 2567.5, 2572.5, 2577.5,
       2582.5, 2587.5, 2592.5, 2597.5, 2602.5, 2607.5, 2612.5, 2617.5, 2622.5,
       2627.5, 2632.5, 2637.5, 2642.5, 2647.5, 2652.5, 2657.5, 2662.5, 2667.5,
       2672.5, 2677.5, 2682.5, 2687.5, 2692.5, 2697.5, 2702.5, 2707.5, 2712.5,
       2717.5, 2722.5, 2727.5, 2732.5, 2737.5, 2742.5, 2747.5, 2752.5, 2757.5,
       2762.5, 2767.5, 2772.5, 2777.5, 2782.5, 2787.5, 2792.5, 2797.5, 2802.5,
       2807.5, 2812.5, 2817.5, 2822.5, 2827.5, 2832.5, 2837.5, 2842.5, 2847.5,
       2852.5, 2857.5, 2862.5, 2867.5, 2872.5, 2877.5, 2882.5, 2887.5, 2892.5,
       2897.5])
Coordinates:
  * time     (time) float64 2.402e+03 2.408e+03 ... 2.892e+03 2.898e+03

### Unfiltered NeverWorld2 data

In [6]:
path = '/glade/p/univ/unyu0004/gmarques/NeverWorld2/baselines/'
chunks = {'time': 1, 'zl':1}

st = xr.open_dataset('%s/%s/static.nc' % (path,run), decode_times=False)
av = xr.open_mfdataset('%s/%s/averages_*.nc' % (path, run), decode_times=False, chunks=chunks, combine='by_coords')

### Choose only 500 last days from `av` data to be consistent with `av_f` data

In [7]:
av = av.isel(time=slice(-100, None))
av.time

<xarray.DataArray 'time' (time: 100)>
array([2402.5, 2407.5, 2412.5, 2417.5, 2422.5, 2427.5, 2432.5, 2437.5, 2442.5,
       2447.5, 2452.5, 2457.5, 2462.5, 2467.5, 2472.5, 2477.5, 2482.5, 2487.5,
       2492.5, 2497.5, 2502.5, 2507.5, 2512.5, 2517.5, 2522.5, 2527.5, 2532.5,
       2537.5, 2542.5, 2547.5, 2552.5, 2557.5, 2562.5, 2567.5, 2572.5, 2577.5,
       2582.5, 2587.5, 2592.5, 2597.5, 2602.5, 2607.5, 2612.5, 2617.5, 2622.5,
       2627.5, 2632.5, 2637.5, 2642.5, 2647.5, 2652.5, 2657.5, 2662.5, 2667.5,
       2672.5, 2677.5, 2682.5, 2687.5, 2692.5, 2697.5, 2702.5, 2707.5, 2712.5,
       2717.5, 2722.5, 2727.5, 2732.5, 2737.5, 2742.5, 2747.5, 2752.5, 2757.5,
       2762.5, 2767.5, 2772.5, 2777.5, 2782.5, 2787.5, 2792.5, 2797.5, 2802.5,
       2807.5, 2812.5, 2817.5, 2822.5, 2827.5, 2832.5, 2837.5, 2842.5, 2847.5,
       2852.5, 2857.5, 2862.5, 2867.5, 2872.5, 2877.5, 2882.5, 2887.5, 2892.5,
       2897.5])
Coordinates:
  * time     (time) float64 2.402e+03 2.408e+03 ... 2.892e+03 2.898e+03
Attributes:
    long_name:       time
    units:           days since 0001-01-01 00:00:00
    cartesian_axis:  T
    calendar_type:   THIRTY_DAY_MONTHS
    calendar:        THIRTY_DAY_MONTHS
    bounds:          time_bnds

### NW2 grid info

In [8]:
from xgcm import Grid

Nx = np.size(st.xh)
Ny = np.size(st.yh)

# symmetric
coords = {
    'X': {'center': 'xh', 'outer': 'xq'},
    'Y': {'center': 'yh', 'outer': 'yq'}
}
metrics = {
    ('X',):['dxCu','dxCv','dxT','dxBu'],
    ('Y',):['dyCu','dyCv','dyT','dyBu'],
    ('X', 'Y'): ['area_t', 'area_u', 'area_v']
}

grid = Grid(st, coords=coords, periodic=['X'])

st['dxT'] = grid.interp(st.dxCu,'X')
st['dyT'] = grid.interp(st.dyCv,'Y', boundary='fill')
st['dxBu'] = grid.interp(st.dxCv,'X')
st['dyBu'] = grid.interp(st.dyCu,'Y',boundary='fill')

grid = Grid(st, coords=coords, periodic=['X'], metrics=metrics)
grid

<xgcm.Grid>
X Axis (periodic, boundary=None):
  * center   xh --> outer
  * outer    xq --> center
Y Axis (not periodic, boundary=None):
  * center   yh --> outer
  * outer    yq --> center

## Compute wind work diagnostics

### with spatial filter

In [9]:
ds_spatial = xr.Dataset()

In [10]:
# wind work on MKE and EKE reservoirs 
# same as in compute_2d_Lorenz_cycle.ipynb and compute_2d_Bleck_cycle.ipynb but not depth-integrated

# non-TWA (compare compute_2d_Lorenz_cycle.ipynb)
ds_spatial['MKE_wind_stress'] = av_f['h'] * (
    grid.interp((av_f['u'] * (av_f['du_dt_str_visc_rem'])).fillna(value=0), 'X', metric_weighted=['X','Y']) 
    + grid.interp(av_f['v'] * (av_f['dv_dt_str_visc_rem']).fillna(value=0), 'Y', metric_weighted=['X','Y']) 
)
    
ds_spatial['EKE_wind_stress'] = av_f['KE_stress'] - ds_spatial['MKE_wind_stress']

# TWA (compare compute_2d_Bleck_cycle.ipynb)
ds_spatial['MKE_TWA_wind_stress'] = (
    grid.interp((av_f['uh'] * st['dxCu'] * av_f['h_du_dt_str']).fillna(value=0), 'X', metric_weighted=['X','Y'])
    + grid.interp((av_f['vh'] * st['dyCv'] * av_f['h_dv_dt_str']).fillna(value=0), 'Y', metric_weighted=['X','Y'])
) / av_f['h'] / st['area_t']

ds_spatial['EKE_TWA_wind_stress'] = av_f['KE_stress'] - ds_spatial['MKE_TWA_wind_stress']

In [11]:
ds_spatial['e'] = av_f['e']

In [12]:
# compute 500 day mean
dst_spatial = ds_spatial.mean(dim='time')

### with temporal filter

In [13]:
namelist = ['h', 'KE_stress', 'e', 'u', 'du_dt_str_visc_rem', 'uh', 'h_du_dt_str', 'v', 'dv_dt_str_visc_rem', 'vh', 'h_dv_dt_str']

In [14]:
av_t = av.mean(dim='time')

In [15]:
dst_temporal = xr.Dataset()

In [16]:
# wind work on MKE and EKE reservoirs 
# as above but for temporal filter

# non-TWA (compare compute_2d_Lorenz_cycle.ipynb)
dst_temporal['MKE_wind_stress'] = av_t['h'] * (
    grid.interp((av_t['u'] * (av_t['du_dt_str_visc_rem'])).fillna(value=0), 'X', metric_weighted=['X','Y']) 
    + grid.interp(av_t['v'] * (av_t['dv_dt_str_visc_rem']).fillna(value=0), 'Y', metric_weighted=['X','Y']) 
)
    
dst_temporal['EKE_wind_stress'] = av_t['KE_stress'] - dst_temporal['MKE_wind_stress']

# TWA (compare compute_2d_Bleck_cycle.ipynb)
dst_temporal['MKE_TWA_wind_stress'] = (
    grid.interp((av_t['uh'] * st['dxCu'] * av_t['h_du_dt_str']).fillna(value=0), 'X', metric_weighted=['X','Y'])
    + grid.interp((av_t['vh'] * st['dyCv'] * av_t['h_dv_dt_str']).fillna(value=0), 'Y', metric_weighted=['X','Y'])
) / av_t['h'] / st['area_t']

dst_temporal['EKE_TWA_wind_stress'] = av_t['KE_stress'] - dst_temporal['MKE_TWA_wind_stress']

In [17]:
dst_temporal['e'] = av_t['e']

## Write to file

In [18]:
nr_days = 5 * len(av_f.time)
nr_days

500

In [19]:
workpath = '/glade/work/noraloose/'

In [23]:
filename_s = '%s/%s/wind_work_diags_spatial_fac%i_500days.nc' %(workpath, run, filter_fac)
filename_s

'/glade/work/noraloose//nw2_0.03125deg_N15_baseline_hmix20/wind_work_diags_spatial_fac32_500days.nc'

In [21]:
dst_spatial

<xarray.Dataset>
Dimensions:              (xh: 1920, yh: 4480, zl: 15, zi: 16)
Coordinates:
  * xh                   (xh) float64 0.01562 0.04688 0.07812 ... 59.95 59.98
  * yh                   (yh) float64 -69.98 -69.95 -69.92 ... 69.92 69.95 69.98
  * zl                   (zl) float64 1.023e+03 1.023e+03 ... 1.028e+03
  * zi                   (zi) float64 1.022e+03 1.023e+03 ... 1.028e+03
Data variables:
    MKE_wind_stress      (zl, yh, xh) float64 dask.array<chunksize=(1, 4480, 1920), meta=np.ndarray>
    EKE_wind_stress      (zl, yh, xh) float64 dask.array<chunksize=(1, 4480, 1920), meta=np.ndarray>
    MKE_TWA_wind_stress  (zl, yh, xh) float64 dask.array<chunksize=(1, 4480, 1920), meta=np.ndarray>
    EKE_TWA_wind_stress  (zl, yh, xh) float64 dask.array<chunksize=(1, 4480, 1920), meta=np.ndarray>
    e                    (zi, yh, xh) float64 dask.array<chunksize=(16, 4480, 1920), meta=np.ndarray>

In [24]:
with ProgressBar():
    dst_spatial.to_netcdf(filename_s)

[########################################] | 100% Completed |  2hr  7min  2.0s


In [27]:
filename_t = '%s/%s/wind_work_diags_temporal_500days.nc' %(workpath, run)
filename_t

'/glade/work/noraloose//nw2_0.03125deg_N15_baseline_hmix20/wind_work_diags_temporal_500days.nc'

In [28]:
dst_temporal

<xarray.Dataset>
Dimensions:              (yh: 4480, zl: 15, xh: 1920, zi: 16)
Coordinates:
  * yh                   (yh) float64 -69.98 -69.95 -69.92 ... 69.92 69.95 69.98
  * zl                   (zl) float64 1.023e+03 1.023e+03 ... 1.028e+03
  * xh                   (xh) float64 0.01562 0.04688 0.07812 ... 59.95 59.98
  * zi                   (zi) float64 1.022e+03 1.023e+03 ... 1.028e+03
Data variables:
    MKE_wind_stress      (zl, yh, xh) float64 dask.array<chunksize=(1, 4480, 1920), meta=np.ndarray>
    EKE_wind_stress      (zl, yh, xh) float64 dask.array<chunksize=(1, 4480, 1920), meta=np.ndarray>
    MKE_TWA_wind_stress  (zl, yh, xh) float64 dask.array<chunksize=(1, 4480, 1920), meta=np.ndarray>
    EKE_TWA_wind_stress  (zl, yh, xh) float64 dask.array<chunksize=(1, 4480, 1920), meta=np.ndarray>
    e                    (zi, yh, xh) float32 dask.array<chunksize=(16, 4480, 1920), meta=np.ndarray>

In [29]:
with ProgressBar():
    dst_temporal.to_netcdf(filename_t)

[########################################] | 100% Completed |  1hr 16min 12.8s
